In [6]:
%matplotlib inline
import numpy
import torch
import random

## Imprementation from scratch

### Synthetic dataset
The input $\mathbf{x}=[x_1, x_2]$ of which both elements are from the normal distribution $\mathcal{N}(0,1)$.
The output $y$ is calculated from
   
   $$y=\mathbf{w}^T \mathbf{x}+b+\epsilon$$

where $\epsilon$ is a Gaussian noise driven from $\mathcal{N}(0,0.01)$, $\mathbf{w}=\begin{bmatrix}2 & -3.4 \end{bmatrix}^T$, and $b=4.2$.

In [70]:
def synthesize_data(w, b, num_examples):
    X = torch.normal(mean=0, std=1, size=(num_examples, len(w)))
    y = torch.matmul(X, w)+b
    epsilon = torch.normal(0, 0.01, size=y.shape)
    y += epsilon
    return X, y.reshape([-1, 1])

In [71]:
true_w = torch.tensor([2,-3.4])
true_b = torch.tensor(4.2)
features, labels = synthesize_data(true_w, true_b, 1000)

### Read the dataset

Here we define a generator that will return us a minibatch of the training set each time.

In [72]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    
    random.shuffle(indices)

    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+
                                                   batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


### Parameter initialization
By default, the `requires_grad`attribiute for a tensor is `False`. So we need to explicitly specify them when we initialize $\mathbf{w}$ and $b$.

In [73]:
w = torch.normal(0,0.01, size = (2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

### Model
The model is defined by:
$$y = \mathbf{w}^T\mathbf{x}+b$$
With a minibatch of inputs, $\mathbf{X}$ is an matrix, so it can be written as:
$$\mathbf{y}=\mathbf{X}\mathbf{w}+b$$

In [74]:
def linreg(X, w, b):
    return torch.mm(X, w)+b

### Loss Function

In [75]:
# def squared_loss(y_hat, y):
#     return (y_hat-y.reshape(y_hat.shape))**2/2
def squared_loss(y_hat, y):  #@save
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

### Optimizer
The updating rule for the parameters are
$$\mathbf{w} = \mathbf{w}-\eta * \mathbf{w}.grad$$
$$b = b-\eta * b.grad$$

Since pytorch will accululate gradients, we need to set the variable manually to zero after we've done a computation. For why Pytorh is doing so plz check this link [Why do we need to set the gradients manually to zero in pytorch?
](https://discuss.pytorch.org/t/why-do-we-need-to-set-the-gradients-manually-to-zero-in-pytorch/4903).

In [76]:
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad/batch_size
            param.grad.zero_()

### Training
The whole process of the traning is:
1. initialize parameters;
2. compute loss function value based on the current parameters; 
3. compute the gradients;
4. update paramters acorrding to the gradients; check if we meet the stopping criteria; if not, go to step 2, else, go to step 5;
5. return the paramters;


In [77]:
lr = 0.03
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        y_hat = linreg(X, w, b)
        sq_loss = squared_loss(y_hat, y)
        sq_loss.sum().backward()
        sgd([w, b], lr, batch_size)
    # after each iteration, print out the traning loss
    with torch.no_grad():
        y_hat = linreg(X, w, b)
        sq_loss = squared_loss(y_hat, y)
        print(f'epoch {epoch + 1}, loss {float(sq_loss.mean()):f}')

epoch 1, loss 0.022369
epoch 2, loss 0.000133
epoch 3, loss 0.000071


In [78]:
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: tensor([ 0.0006, -0.0002], grad_fn=<SubBackward0>)
error in estimating b: tensor([0.0008], grad_fn=<SubBackward0>)


## Implementation using Pytorch

### Synthetic Dataset
The synthetic dataset we generated here is the same as the one we generated before.

In [79]:
true_w = torch.tensor([2,-3.4])
true_b = torch.tensor(4.2)
features, labels = synthesize_data(true_w, true_b, 1000)

### Data Iterator
Here we use `torch.utils.data.TensorDataset` to wrap our synthetic dataset. 

In [89]:
def load__array(data_arrays, batch_size, is_train=True):
    dataset = torch.utils.data.TensorDataset(*data_arrays)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle=is_train)
    return dataloader

### Model
The model is the same.
The model is defined by:
$$y = \mathbf{w}^T\mathbf{x}+b$$
With a minibatch of inputs, $\mathbf{X}$ is an matrix, so it can be written as:
$$\mathbf{y}=\mathbf{X}\mathbf{w}+b$$

This time we are using the `nn.Sequential` and `nn.Linear`。

In [83]:
lin_reg = torch.nn.Sequential(torch.nn.Linear(2,1))

### Initialize Parameters

### Loss Function

In [85]:
loss = torch.nn.MSELoss()

### Optimizer


In [87]:
trainer = torch.optim.SGD(lin_reg.parameters(), lr=0.03)

### Training

In [90]:
batch_size = 10
num_epochs = 3

for i in range(num_epochs):
    for X, y in load__array([features, labels], batch_size, is_train=True):
        l = loss(lin_reg(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(lin_reg(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 3, loss 0.000270
epoch 3, loss 0.000098
epoch 3, loss 0.000098


In [92]:
w = lin_reg[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = lin_reg[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-0.0005,  0.0004])
error in estimating b: tensor([-0.0002])
